<a href="https://colab.research.google.com/github/sophiawang705/AISF-Bipedal-Walker/blob/main/Fellowship.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install gymnasium[box2d] stable_baselines3 shimmy

  Using cached box2d-py-2.3.5.tar.gz (374 kB)
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  error: subprocess-exited-with-error
  
  × Building wheel for box2d-py (pyproject.toml) did not run successfully.
  │ exit code: 1
  ╰─> No available output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for box2d-py
Failed to build box2d-py
error: failed-wheel-build-for-install

× Failed to build installable wheels for some pyproject.toml based projects
╰─> box2d-py


In [1]:
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize
from stable_baselines3.common.evaluation import evaluate_policy
import os

# 1. Setup Environment
env_id = "BipedalWalker-v3"
# For 'hardcore', change to "BipedalWalkerHardcore-v3"
env = gym.make(env_id, render_mode="rgb_array")
env = DummyVecEnv([lambda: env])

# CRITICAL: Normalize observations and rewards for stability
env = VecNormalize(env, norm_obs=True, norm_reward=True, clip_obs=10.)

# 2. Define the Model (MLP Policy)
model = PPO(
    "MlpPolicy",
    env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01, # Helps with exploration
    tensorboard_log="./ppo_walker_tensorboard/"
)

# 3. Train (Start with 100k steps for baseline, target 1M+ for solution)
print("Training...")
model.learn(total_timesteps=100000)

# 4. Save the model and statistics
model.save("ppo_bipedal_walker")
env.save("vec_normalize.pkl")

Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyPacked has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type SwigPyObject has no __module__ attribute
<frozen importlib._bootstrap>:488: DeprecationWarning: builtin type swigvarlink has no __module__ attribute
/usr/local/lib/python3.12/dist-packages/pygame/pkgdata.py:25: DeprecationWarning: pkg_resources is deprecated as an API. See https://setuptools.pypa.io/en/latest/pkg_resources.html
  from pkg_resources import resource_stream, resource_exists
/usr/local/lib/python3.12/dist-packages/pkg_resources/__init__.py:315

Using cpu device
Training...
Logging to ./ppo_walker_tensorboard/PPO_1


/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


-----------------------------
| time/              |      |
|    fps             | 611  |
|    iterations      | 1    |
|    time_elapsed    | 3    |
|    total_timesteps | 2048 |
-----------------------------
-----------------------------------------
| time/                   |             |
|    fps                  | 567         |
|    iterations           | 2           |
|    time_elapsed         | 7           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.011883976 |
|    clip_fraction        | 0.0827      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.72       |
|    explained_variance   | -0.102      |
|    learning_rate        | 0.0003      |
|    loss                 | 0.196       |
|    n_updates            | 10          |
|    policy_gradient_loss | -0.0147     |
|    std                  | 1.02        |
|    value_loss           | 1.29        |
----------------------------------

In [3]:
import gymnasium as gym
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(model_path, env_id, video_length=500, prefix="walker-video"):
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Load normalization stats from your training
    eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)
    # Stop updating normalization stats during evaluation
    eval_env.training = False
    eval_env.norm_reward = False

    eval_env = VecVideoRecorder(eval_env, "./videos/",
                                record_video_trigger=lambda x: x == 0,
                                video_length=video_length,
                                name_prefix=prefix)

    model = PPO.load(model_path, env=eval_env)
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, dones, info = eval_env.step(action)

    eval_env.close()

# Run the recording
record_video("ppo_bipedal_walker", "BipedalWalker-v3")

Saving video to /content/videos/walker-video-step-0-to-step-500.mp4


/usr/local/lib/python3.12/dist-packages/moviepy/config_defaults.py:47: SyntaxWarning: invalid escape sequence '\P'
  IMAGEMAGICK_BINARY = r"C:\Program Files\ImageMagick-6.8.8-Q16\magick.exe"
/usr/local/lib/python3.12/dist-packages/jupyter_client/session.py:203: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  return datetime.utcnow().replace(tzinfo=utc)


Moviepy - Building video /content/videos/walker-video-step-0-to-step-500.mp4.
Moviepy - Writing video /content/videos/walker-video-step-0-to-step-500.mp4



Moviepy - Done !
Moviepy - video ready /content/videos/walker-video-step-0-to-step-500.mp4


In [5]:
from IPython.display import HTML
from base64 import b64encode

def show_video(video_path):
    mp4 = open(video_path,'rb').read()
    data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
    return HTML(f"""
    <video width=400 controls>
          <source src="{data_url}" type="video/mp4">
    </video>
    """)

show_video('/content/videos/walker-video-step-0-to-step-500.mp4')

In [6]:
import os
import gymnasium as gym
from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import DummyVecEnv, VecNormalize, VecMonitor
from stable_baselines3.common.evaluation import evaluate_policy

env_id = "BipedalWalker-v3"
os.makedirs("models", exist_ok=True)

# --- TRAIN ENV (NO RENDER) ---
train_env = DummyVecEnv([lambda: gym.make(env_id)])
train_env = VecMonitor(train_env)

# Normalize obs + rewards for training stability
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True, clip_obs=10.0, clip_reward=10.0)

model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01,
    tensorboard_log="./ppo_walker_tensorboard/",
)

print("Training...")
model.learn(total_timesteps=100_000_000)

model.save("models/ppo_bipedalwalker_vecnorm")
train_env.save("models/vecnormalize.pkl")

# --- EVAL ENV (NO RENDER) ---
eval_env = DummyVecEnv([lambda: gym.make(env_id)])
eval_env = VecMonitor(eval_env)
eval_env = VecNormalize.load("models/vecnormalize.pkl", eval_env)

# IMPORTANT: evaluation mode
eval_env.training = False
eval_env.norm_reward = False

mean_reward, std_reward = evaluate_policy(model, eval_env, n_eval_episodes=5, deterministic=True)
print(f"Eval mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

train_env.close()
eval_env.close()


Using cpu device
Training...
Logging to ./ppo_walker_tensorboard/PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 339      |
|    ep_rew_mean     | -112     |
| time/              |          |
|    fps             | 760      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 355         |
|    ep_rew_mean          | -111        |
| time/                   |             |
|    fps                  | 624         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009012805 |
|    clip_fraction        | 0.0805      |
|    clip_range           | 0.2         |
|    entropy_loss         | -5.72       |
|

KeyboardInterrupt: 

In [ ]:
import gymnasium as gym
from stable_baselines3.common.vec_env import VecVideoRecorder, DummyVecEnv

def record_video(model_path, env_id, video_length=500, prefix="walker-video"):
    eval_env = DummyVecEnv([lambda: gym.make(env_id, render_mode="rgb_array")])
    # Load normalization stats from your training
    eval_env = VecNormalize.load("vec_normalize.pkl", eval_env)
    # Stop updating normalization stats during evaluation
    eval_env.training = False
    eval_env.norm_reward = False

    eval_env = VecVideoRecorder(eval_env, "./videos/",
                                record_video_trigger=lambda x: x == 0,
                                video_length=video_length,
                                name_prefix=prefix)

    model = PPO.load(model_path, env=eval_env)
    obs = eval_env.reset()
    for _ in range(video_length):
        action, _states = model.predict(obs, deterministic=True)
        obs, rewards, dones, info = eval_env.step(action)

    eval_env.close()

# Run the recording
record_video("ppo_bipedal_walker", "BipedalWalker-v3")

In [7]:
import pandas as pd
import matplotlib.pyplot as plt
from stable_baselines3.common.results_plotter import load_results, ts2xy

def plot_results(log_folder, title='Learning Curve'):
    x, y = ts2xy(load_results(log_folder), 'timesteps')
    y_moving_average = pd.Series(y).rolling(window=10).mean()

    plt.figure(figsize=(10, 5))
    plt.plot(x, y, alpha=0.2, label='Episode Reward')
    plt.plot(x, y_moving_average, label='Moving Average (10 eps)')
    plt.xlabel('Number of Timesteps')
    plt.ylabel('Rewards')
    plt.title(title)
    plt.legend()
    plt.show()

plot_results("./ppo_walker_tensorboard/PPO_2")

LoadMonitorResultsError: No monitor files of the form *monitor.csv found in ./ppo_walker_tensorboard/PPO_2

In [11]:
import os
import numpy as np
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecNormalize,
    VecMonitor,
    VecEnvWrapper,
)
from stable_baselines3.common.evaluation import evaluate_policy


# -----------------------------
# Action Smoothing Wrapper
# -----------------------------
class ActionSmoothingVecEnv(VecEnvWrapper):
    """
    Smooth actions over time:
      a_smooth[t] = alpha * a_smooth[t-1] + (1 - alpha) * a_raw[t]
    """

    def __init__(self, venv, alpha=0.90, clip_low=-1.0, clip_high=1.0):
        super().__init__(venv)
        assert 0.0 <= alpha < 1.0
        self.alpha = float(alpha)
        self.clip_low = float(clip_low)
        self.clip_high = float(clip_high)
        self._last_action = None

    def reset(self):
        obs = self.venv.reset()
        action_shape = (self.num_envs,) + self.action_space.shape
        self._last_action = np.zeros(action_shape, dtype=np.float32)
        return obs

    def step_async(self, actions):
        actions = np.asarray(actions, dtype=np.float32)

        if self._last_action is None:
            action_shape = (self.num_envs,) + self.action_space.shape
            self._last_action = np.zeros(action_shape, dtype=np.float32)

        smoothed = self.alpha * self._last_action + (1.0 - self.alpha) * actions
        smoothed = np.clip(smoothed, self.clip_low, self.clip_high)
        self._last_action = smoothed

        self.venv.step_async(smoothed)

    def step_wait(self):
        return self.venv.step_wait()


# -----------------------------
# Setup
# -----------------------------
env_id = "BipedalWalker-v3"
os.makedirs("models", exist_ok=True)

# -----------------------------
# TRAIN ENV (NO RENDER)
# -----------------------------
train_env = DummyVecEnv([lambda: gym.make(env_id)])
train_env = VecMonitor(train_env)

# ⭐ Unique contribution
train_env = ActionSmoothingVecEnv(train_env, alpha=0.90)

# ⭐ Stability
train_env = VecNormalize(
    train_env,
    norm_obs=True,
    norm_reward=True,
    clip_obs=10.0,
    clip_reward=10.0,
)

# -----------------------------
# PPO Model
# -----------------------------
model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=64,
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.01,
    target_kl=0.03,
    tensorboard_log="./ppo_walker_tensorboard/",
)

print("Training...")
model.learn(total_timesteps=1_000_000)

# -----------------------------
# Save
# -----------------------------
model.save("models/ppo_bipedalwalker_vecnorm_smooth")
train_env.save("models/vecnormalize_smooth.pkl")

# -----------------------------
# EVAL ENV (NO RENDER)
# -----------------------------
eval_env = DummyVecEnv([lambda: gym.make(env_id)])
eval_env = VecMonitor(eval_env)
eval_env = VecNormalize.load("models/vecnormalize_smooth.pkl", eval_env)

eval_env.training = False
eval_env.norm_reward = False

mean_reward, std_reward = evaluate_policy(
    model,
    eval_env,
    n_eval_episodes=10,
    deterministic=True,
)

print(f"Eval mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

train_env.close()
eval_env.close()


Streaming output truncated to the last 5000 lines.
|    ep_rew_mean          | -11.6       |
| time/                   |             |
|    fps                  | 491         |
|    iterations           | 264         |
|    time_elapsed         | 1100        |
|    total_timesteps      | 540672      |
| train/                  |             |
|    approx_kl            | 0.011621077 |
|    clip_fraction        | 0.119       |
|    clip_range           | 0.2         |
|    entropy_loss         | -7.06       |
|    explained_variance   | 0.654       |
|    learning_rate        | 0.0003      |
|    loss                 | -0.0798     |
|    n_updates            | 2612        |
|    policy_gradient_loss | -0.0199     |
|    std                  | 1.49        |
|    value_loss           | 0.000898    |
-----------------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 1.47e+03    |
|    ep_rew_mean         

In [12]:
from IPython.display import HTML
from base64 import b64encode

# Path to the video you just generated
video_path = '/content/videos/walker-video-step-0-to-step-500.mp4'

mp4 = open(video_path,'rb').read()
data_url = "data:video/mp4;base64," + b64encode(mp4).decode()

HTML(f"""
<video width=600 controls>
      <source src="{data_url}" type="video/mp4">
</video>
""")

In [ ]:
import os
import numpy as np
import gymnasium as gym

from stable_baselines3 import PPO
from stable_baselines3.common.vec_env import (
    DummyVecEnv,
    VecNormalize,
    VecMonitor,
    VecEnvWrapper,
)
from stable_baselines3.common.evaluation import evaluate_policy

# -----------------------------
# Action Smoothing Wrapper (Modified for faster learning)
# -----------------------------
class ActionSmoothingVecEnv(VecEnvWrapper):
    def __init__(self, venv, alpha=0.70): # Reduced from 0.90 to allow faster feedback
        super().__init__(venv)
        self.alpha = float(alpha)
        self._last_action = None

    def reset(self):
        obs = self.venv.reset()
        self._last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        return obs

    def step_async(self, actions):
        if self._last_action is None:
            self._last_action = np.zeros((self.num_envs,) + self.action_space.shape, dtype=np.float32)
        # alpha * old + (1-alpha) * new
        smoothed = self.alpha * self._last_action + (1.0 - self.alpha) * actions
        self._last_action = np.clip(smoothed, -1.0, 1.0)
        self.venv.step_async(self._last_action)

    def step_wait(self):
        return self.venv.step_wait()

# -----------------------------
# Setup & Training
# -----------------------------
env_id = "BipedalWalker-v3"
os.makedirs("models", exist_ok=True)
os.makedirs("videos", exist_ok=True)

# Create Training Env
def make_env():
    return gym.make(env_id)

train_env = DummyVecEnv([make_env])
train_env = VecMonitor(train_env)
train_env = ActionSmoothingVecEnv(train_env, alpha=0.70) # Pivot: Faster feedback
train_env = VecNormalize(train_env, norm_obs=True, norm_reward=True)

# -----------------------------
# PPO Model (Optimized Hyperparameters)
# -----------------------------
model = PPO(
    "MlpPolicy",
    train_env,
    verbose=1,
    learning_rate=3e-4,
    n_steps=2048,
    batch_size=128,      # Increased for stability
    n_epochs=10,
    gamma=0.99,
    gae_lambda=0.95,
    ent_coef=0.02,       # Pivot: Higher entropy forces exploration out of -113
    clip_range=0.2,
    tensorboard_log="./ppo_walker_tensorboard/",
)

print("🚀 Training starting... Look for reward to break -100")
model.learn(total_timesteps=1_000_000)

# Save
model.save("models/ppo_walker_final")
train_env.save("models/vecnormalize_final.pkl")

# -----------------------------
# VIDEO GENERATION (The "Application Clip")
# -----------------------------
print("📹 Generating video of best performance...")
video_env_raw = gym.make(env_id, render_mode="rgb_array")
video_env_raw = gym.wrappers.RecordVideo(video_env_raw, "./videos/", name_prefix="final_submission")

video_env = DummyVecEnv([lambda: video_env_raw])
video_env = VecNormalize.load("models/vecnormalize_final.pkl", video_env)
video_env.training = False
video_env.norm_reward = False

obs = video_env.reset()
for _ in range(1000):
    action, _ = model.predict(obs, deterministic=True)
    obs, reward, done, info = video_env.step(action)
    if done: break

video_env.close()
print("Done! Download your video from the 'videos' folder on the left.")

Using cpu device
🚀 Training starting... Look for reward to break -100
Logging to ./ppo_walker_tensorboard/PPO_5
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 270      |
|    ep_rew_mean     | -116     |
| time/              |          |
|    fps             | 689      |
|    iterations      | 1        |
|    time_elapsed    | 2        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 576         |
|    ep_rew_mean          | -91.2       |
| time/                   |             |
|    fps                  | 605         |
|    iterations           | 2           |
|    time_elapsed         | 6           |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.008731916 |
|    clip_fraction        | 0.068       |
|    clip_range           | 0.2         |
| 